In [1]:
# Standard library
import os
from pathlib import Path
import shlex

# Third party
import hydra
import rich

# First Party
BASE_PATH = Path("/home/mila/g/gagnonju/DPR/")
CONF_PATH = BASE_PATH/"conf"
OUTPUTS = Path("/home/mila/g/gagnonju/IteratedDecoding/outputs/")
OUT_FILE = Path(
    "/home/mila/g/gagnonju/DPR/outputs/integrated_script_attempt/"
)
 
os.chdir(BASE_PATH)
import common_retriever

2021-09-09 00:51:13,811 [INFO] faiss.loader: Loading faiss with AVX2 support.
2021-09-09 00:51:13,813 [INFO] faiss.loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2021-09-09 00:51:13,814 [INFO] faiss.loader: Loading faiss.
2021-09-09 00:51:14,032 [INFO] faiss.loader: Successfully loaded faiss.
[140236006750016] 2021-09-09 00:51:19,861 [INFO] common_retriever: Checking versions...
[140236006750016] 2021-09-09 00:51:19,982 [INFO] transformers.file_utils: PyTorch version 1.7.0 available.
[140236006750016] 2021-09-09 00:51:24,472 [INFO] common_retriever: All version checks passed.


In [ ]:
with hydra.initialize_config_dir(
    config_dir=str(CONF_PATH), 
):
    cfg = hydra.compose(
        config_name="dense_retriever",
        overrides=[
        f"out_file={shlex.quote(str(OUT_FILE))}",
    ])
    
    retriever, all_passages = common_retriever.make_retriever(cfg)
    questions, question_answers, special_query_token = common_retriever.load_data(cfg)
    n_docs = cfg.n_docs

/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dense_retriever': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'ctx_sources/default_sources': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/home/mila/g/gagnonju/.anaconda3/lib/python3.8/site-packages/hydra/core/default_element.py:122: UserWarning: In 'datasets/retriever_default': Usage of deprecated keyword in package header '# @package _group_'.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/changes_to_package_header for more information
  deprecation_warning(
/home/mila/g/gagnonju/.anaconda3/lib/pyth

Done with the reading loop.

[140236006750016] 2021-09-09 00:55:53,995 [INFO] common_retriever: Done loading passages.
[140236006750016] 2021-09-09 00:55:53,996 [INFO] common_retriever: Loading index.
[140236006750016] 2021-09-09 00:55:53,997 [INFO] common_retriever: Index class <class 'dpr.indexer.faiss_indexers.DenseFlatIndexer'> 
[140236006750016] 2021-09-09 00:55:53,999 [INFO] common_retriever: Done loading index.
[140236006750016] 2021-09-09 00:55:54,000 [INFO] common_retriever: Loading retriever.
[140236006750016] 2021-09-09 00:55:54,001 [INFO] common_retriever: Loaded retriever.
[140236006750016] 2021-09-09 00:55:54,004 [INFO] common_retriever: Index path: /home/mila/g/gagnonju/DPR/dpr/downloads/indexes/single/nq/full
[140236006750016] 2021-09-09 00:55:54,004 [INFO] root: Loading index from /home/mila/g/gagnonju/DPR/dpr/downloads/indexes/single/nq/full
[140236006750016] 2021-09-09 00:57:17,079 [INFO] root: Loaded index of type <class 'faiss.swigfaiss.IndexFlat'> and size 21015320
[140236006750016] 2021-09-0

In [ ]:
def retrieve(retriever, all_passages, question, question_answers, special_query_token, n_docs):
    if len(all_passages) == 0:
        raise RuntimeError("No passages data found.")
        
    ###########################################################################
    # Get top k results.
    ###########################################################################
    LOGGER.info("Using special token %s", special_query_token)
    questions_tensor = retriever.generate_question_vectors(
        questions, 
        query_token=question_answers,
    )
    
    LOGGER.info(f"get_top_docs: Starting.")
    top_ids_and_scores = retriever.get_top_docs(
        questions_tensor.numpy(), 
        n_docs,
    )
    LOGGER.info("get_top_docs: Done.")
    
    return  top_ids_and_scores
    
top_ids_and_scores = retrieve(retriever, all_passages, question, quetion_answers, special_query_token, n_docs)

In [ ]:
CONCATENATION_TECHNIQUES = dict()
MAX_LOOP_N = 15
MODEL_PATH = None
CONCATENATION_TECHNIQUE = None
BATCH_SIZE_QUESTIONS = 10
OUTPUT_FILES_ROOT = None

make_batches = more_itertools.ichunked


def write_contexts(
    all_contexts, 
    context_ids, 
    output_files_root, 
    loop_i,
):
    with open(
        output_files_root/f"contexts_{loop_i}.txt", 
        "a",
    ) as f_out:
        for context_id in context_ids:
            f_out.write(all_contexts[context_id].strip() + "\n")

def write_generations(
    generated_text,
    output_files_root, 
    loop_i,
):
    with open(
        output_files_root/f"generated_{loop_i}.txt", 
        "a",
    ) as f_out:
        text = "\n".join((x.strip() for x in generated_text))
        f_out.write(text)

def inference(
    all_contexts,
    max_loop_n=MAX_LOOP_N, 
    model_path=MODEL_PATH, 
    concatenation_technique=CONCATENATION_TECHNIQUE,
    output_files_root=OUTPUT_FILES_ROOT,
):
    """
    We currently do 
    
    for batch_questions in questions:
        for loop_i in range(max_loop_n)
            encode_context_to_gen
            (load generator to faster memory)
            generate(batch)
            decode_text_from_gen
            
            encode_text_to_retriever
            (load retriever to faster memory)
            retrieve(batch)
            decode_text_from_retriever
    
    We could do
    
    for loop_i in range(max_loop_n)
        # Parallelize as needed if helpful
        # num_questions x num_beams to do
        for batch_questions in zip(
            retrieved_contexts
        ):
            encode_contexts_to_gen
        
        (imaginary barrier)
        (load generator to faster memory)    
        for batch_questions in questions:
            top_beam_ids, top_beam_ppls = generate(batch)
            
        # Parallelize as needed if helpful
        for batch_questions in questions:
            decode_text_from_gen
            
        # Parallelize as needed if helpful
        for batch_questions in questions:
            encode_text_to_retriever

        (imaginary barrier)
        # Parallelize as needed if helpful
        for batch_questions in questions:
            concatenate_contexts

        (imaginary barrier)
        (load retriever to faster memory)
        for batch_questions in questions:            
            retrieve(batch)
        
        # Parallelize as needed if helpful
        for batch_questions in questions:            
            decode_text_from_retriever
    
    Much better for GPU memory locality,
    worse for total memory use. Would maybe allow
    for slightly larger batches.
    
    I think that the fact that we don't need the 
    GPU results right away makes it async and faster
    
    """
    output_files_root = pathlib.Path(output_files_root)
    model = load_model(model_path)
    concatenation_technique = CONCATENATION_TECHNIQUES[
        concatenation_technique
    ]
    question_batches = make_batches(questions)
    
    context_ids_accum = [[] for _ in range(max_loop_n)]
    generation_text_accum = [[] for _ in range(max_loop_n)]
    
    for question_batch in question_batches:
        retrieval_query = question
        all_generations = []
        contexts = retrieve(retrieval_query)

        for loop_i in range(max_loop_n):
            top_beams_ids, top_beams_ppl = generate(
                question, contexts,
            )
            
            # Detokenize top_beams_ids.
            # We do this here because we don't have the choice?
            # Maybe we do though.
            # batch_size x num_beams x (variable num_words)
            top_beams_text = []
            for unit in top_beams_ids:
                top_beams_text.append([
                    tokenizer_generator.decode(ids_beam) 
                    for ids_beam in unit
                ])
            
            # We likely have to decode and re-encode here
            retrieval_query = concatenation_technique(
                question, 
                top_beams_text,
                top_beams_ppl,
                all_results, 
                loop_i,
            )
            
            context_ids = retrieve(retrieval_query)            
            generation_text_accum[loop_i].append(top_beams_text)
            context_ids_accum[loop_i].append(contexts)
    
    
    # The following are bad, 
    context_text_accum = [[] for _ in range(max_loop_n)]

    
    for i, batch in enumerate(context_ids_accum):
        for id_ in batch:
            context_text_accum[i].append(all_contexts[id_])
    
    # [Validate somewhere]
    
#             write_contexts(
#                 all_contexts, 
#                 context_ids, 
#                 output_files_root, 
#                 loop_i,
#             )
            
#             write_generations(
#                 generated_text,
#                 output_files_root,
#                 loop_i,
#             )
            
        